In [4]:
import os
import csv
import cv2
import glob
from PIL import Image
from datetime import datetime, timedelta

In [5]:
key_files = glob.glob('key/*/*.csv')

In [8]:
key_files.sort()key_files.sort()
key_files

['key/402_2/SEN3_00066667BB2C_25590804_140000.sendat.csv',
 'key/402_2/SEN3_00066667BB2C_25590804_150000.sendat.csv',
 'key/402_2/SEN3_00066667BB2C_25590804_160000.sendat.csv',
 'key/402_2/SEN3_00066667BB2C_25590804_170000.sendat.csv',
 'key/402_2/SEN3_00066667BB2C_25590804_180000.sendat.csv',
 'key/402_2/SEN3_00066667BB2C_25590804_190000.sendat.csv',
 'key/402_2/SEN3_00066667BB2C_25590804_200000.sendat.csv',
 'key/402_2/SEN3_00066667BB2C_25590804_210000.sendat.csv',
 'key/402_2/SEN3_00066667BB2C_25590804_220000.sendat.csv',
 'key/402_2/SEN3_00066667BB2C_25590804_230000.sendat.csv',
 'key/402_2/SEN3_00066667BB2C_25590805_000000.sendat.csv',
 'key/402_2/SEN3_00066667BB2C_25590805_010000.sendat.csv',
 'key/402_2/SEN3_00066667BB2C_25590805_020000.sendat.csv',
 'key/402_2/SEN3_00066667BB2C_25590805_030000.sendat.csv',
 'key/402_2/SEN3_00066667BB2C_25590805_040000.sendat.csv',
 'key/402_2/SEN3_00066667BB2C_25590805_050000.sendat.csv',
 'key/402_2/SEN3_00066667BB2C_25590805_060000.sendat.csv

In [ ]:
room_number = key_file.split('/')[1][:3]
csv_date = parse_date(key_file)
next_date = csv_date - timedelta(days=1)
m = csv_date.month
mdd = '{}{:02d}'.format(csv_date.month,csv_date.day)
mdd2 = '{}{:02d}'.format(next_date.month,next_date.day)
path = '/Volumes/MIMAMORI_101/FFOutput/video/{}-20160{}*.mp4'.format(room_number,mdd)
path2 = '/Volumes/MIMAMORI_101/FFOutput/video/{}-20160{}*.mp4'.format(room_number,mdd2)
videofiles = glob.glob(path) + glob.glob(path2)


In [ ]:
test_key = key_files[0]

In [ ]:
with open(test_key, 'r') as csv_file:
    reader = csv.reader(csv_file)
    csv_data = list(reader)

In [ ]:
video_dir = '/Volumes/MIMAMORI_101/FFOutput/video'
video_files = [os.path.join(video_dir, file) for file in os.listdir(video_dir) if file.endswith('mp4')]
cap = cv2.VideoCapture(video_dir + video_files[0])

In [ ]:
def getFrame(myFrameNumber = 0):
    '''return cv2 frame at frame number'''
    cap.set(cv2.CAP_PROP_POS_AVI_RATIO,1)
    totalFrames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    if myFrameNumber >= 0 and myFrameNumber <= totalFrames:
        cap.set(cv2.CAP_PROP_POS_FRAMES,myFrameNumber)
    print(totalFrames)
    return totalFrames

# Remove duplicate tag

In [ ]:
def clean_data(csv_data):
    temp = csv_data.copy()
    temp.sort(key = lambda x: int(x[0]))
    for i in range(len(temp)):
        try:
            while temp[i][1] == temp[i+1][1]:
                if i+1 == len(temp) -1: 
                    break
                del(temp[i+1])
        except Exception as e:
            pass
    return temp

In [ ]:
video_file = video_files[0]

# Parse video date

In [ ]:
def parse_date(filePath):
    '''parse video date from filename'''
    filePath = os.path.basename(filePath)
    if filePath.endswith('csv'):
        date_str = filePath[18:33]
        csv_date = datetime.strptime(date_str, '%Y%m%d_%H%M%S')
        csv_date  = csv_date.replace(year=csv_date.year-543)
        return csv_date
    else:
        date_str = filePath[4:-4]
        video_date = datetime.strptime(date_str, '%Y%m%d_%H-%M-%S')
        return video_date
        

In [ ]:
def get_delta_frame(csv_date, video_date):
    delta = csv_date - video_date
    delta_frame = delta.total_seconds() * 30
    return delta_frame

In [ ]:
def isBetween(csv_date, videofile):
    start = parse_date(videofile)
    if csv_date < start:
        return False
    
    cap = cv2.VideoCapture(video_file)
    cap.set(cv2.CAP_PROP_POS_AVI_RATIO,1)
    duration = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000
    end = start + timedelta(seconds=duration)

    if start <= csv_date <= end:
        print(start,csv_date,end)
        return True
    else:
        return False

# Get Video

In [ ]:
key_file = key_files[2]
video_file = 'video/402-20160804_18-24-05.mp4'

csv_date = parse_date(key_file)

cap = cv2.VideoCapture(video_file)
cap.set(cv2.CAP_PROP_POS_AVI_RATIO,1)
duration = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000
start = parse_date(video_file)
end = start + timedelta(seconds=duration)

print(csv_date)
print(start,end,duration)
start <= csv_date <= end

frame = getFrame(get_delta_frame(csv_date, parse_date(video_file)) + int(tag[0]))
img = cap.read()[1]
Image.fromarray(img)

In [2]:
total_tag = 0
for key_file in key_files:
    with open(key_file, 'r') as csv_file:
        reader = csv.reader(csv_file)
        csv_data = list(reader)
    temp = csv_data.copy()
    temp = clean_data(temp)
    csv_date = parse_date(key_file)
    
    #get videofiles
    room_number = key_file.split('/')[1][:3]
    next_date = csv_date - timedelta(days=1)
    m = csv_date.month
    mdd = '{}{:02d}'.format(csv_date.month,csv_date.day)
    mdd2 = '{}{:02d}'.format(next_date.month,next_date.day)
    path = '/Volumes/MIMAMORI_101/FFOutput/video/{}-20160{}*.mp4'.format(room_number,mdd)
    path2 = '/Volumes/MIMAMORI_101/FFOutput/video/{}-20160{}*.mp4'.format(room_number,mdd2)
    videofiles = glob.glob(path) + glob.glob(path2)
    
    for i, tag in enumerate(temp):
        if int(tag[1]) == 3 and i != len(temp)-1:
            total_tag += 1
            #get Video
            csv_date_new = csv_date + timedelta(seconds=int(tag[0])/30)
            if not isBetween(csv_date_new, video_file):
                for video_file in video_files:
                    if isBetween(csv_date_new, video_file):
                        break
            try:
                #get frame
                cap = cv2.VideoCapture(video_file)
                frame = getFrame(get_delta_frame(csv_date, parse_date(video_file)) + int(tag[0]))
                img = cap.read()[1]
                Image.fromarray(img).show()
                cap.release()
                new_tag = input('Enter new tag@{}: '.format(csv_date))
                temp[i][1] = new_tag
                if i != 0 and len(temp[i-1][1]) == 2:
                    if temp[i-1][1][0] == new_tag:
                        temp[i-1][1] = new_tag    
                    else:
                        temp[i-1][1] = temp[i-1][1][0] + new_tag
                if i != len(temp)-1 and len(temp[i+1][1]) == 2:
                    if temp[i+1][1][1] == new_tag:
                        temp[i+1][1] = new_tag    
                    else:
                        temp[i+1][1] = new_tag + temp[i+1][1][1]
            except Exception:
                pass

    temp = clean_data(temp)
    print(temp,key_file)
print(total_tag)

NameError: name 'key_files' is not defined

In [1]:
temp

NameError: name 'temp' is not defined